In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
#import statements
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

from transformers import AutoConfig, AutoTokenizer, TFAutoModel

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/config.json
Model config BertConfig {
  "_name_or_path": "law-ai/InLegalBERT",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eo

In [ ]:
config = AutoConfig.from_pretrained("law-ai/InLegalBERT")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/config.json
Model config BertConfig {
  "_name_or_path": "law-ai/InLegalBERT",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
config

BertConfig {
  "_name_or_path": "law-ai/InLegalBERT",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:

# Load the data
data = pd.read_csv('final_df.csv')

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)


classes = data.label.unique().tolist()
print(classes)

['PRECEDENT', 'RLC', 'REASONING', 'FACTS', 'ARG', 'JUDGE', 'NAME', 'CITATION', 'COUNSEL', 'RPC', 'STATUTE']


In [ ]:
#drop all the none values
data = data.dropna()

# Split the data into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
train, valid = train_test_split(train, test_size=0.25, random_state=42)

print(train.shape, valid.shape, test.shape)

(1370, 2) (457, 2) (457, 2)


In [ ]:
train = train[['sentence','label']]

In [ ]:
train.head()

,sentence,label
1604,"In the present case, the learned Judge who tri...",REASONING
1743,There is no question of any capital or other e...,REASONING
872,It is this sub clause which the appellant invo...,ARG
1361,Raghbir Singh (amicus curiae) for the petition...,COUNSEL
1568,"Indeed, the heading of Part III is ""Fundamenta...",REASONING


In [ ]:
test = test[['sentence','label']]

In [ ]:
test.head()

,sentence,label
383,The judgment of the Court was deliv ered by FA...,JUDGE
2187,We are here concerned with the latter aspect o...,RPC
2243,"On the 28th of March, 1940, the mortgagee assi...",FACTS
1977,"It was argued that ""public safety"" and ""mainte...",ARG
810,They are framed under section 22 (1) (f) (i) (...,REASONING


In [ ]:
#building model class using TFAutoModel class
max_seq_len = 128
input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
bert_model = TFAutoModel.from_pretrained(pretrained_model_name_or_path='law-ai/InLegalBERT', config=config, from_pt=True)
bert_output = bert_model(input_ids)

bert_output_cls = bert_output.pooler_output


cls_out = keras.layers.Dropout(0.5)(bert_output_cls)
logits = keras.layers.Dense(units=11, activation="softmax")(cls_out)

model = keras.Model(inputs=input_ids, outputs=logits)
model.build(input_shape=(None, max_seq_len))

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--law-ai--InLegalBERT/snapshots/b5ecfed8ed6cf9d25a3cb8225a8c52f161f7401a/pytorch_model.bin
PyTorch checkpoint contains 133,578,358 parameters
Loaded 109,482,240 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with anoth

In [ ]:
#preprocessing
class Classifier:
  DATA_COLUMN = "sentence"
  LABEL_COLUMN = "label"

  def __init__(self, train, test, tokenizer, classes, max_seq_len=128):
    self.tokenizer = tokenizer
    self.max_seq_len = 128
    self.classes = classes

    train, test = map(lambda df: df.reindex(df[Classifier.DATA_COLUMN].str.len().sort_values().index), [train, test])

    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

  def _prepare(self, df):
    x, y = [], []

    for _, row in tqdm(df.iterrows()):
      text, label = row[Classifier.DATA_COLUMN], row[Classifier.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [ ]:
classes = train.label.unique().tolist()
print(classes)

['REASONING', 'ARG', 'COUNSEL', 'FACTS', 'NAME', 'PRECEDENT', 'STATUTE', 'RPC', 'CITATION', 'RLC', 'JUDGE']


In [ ]:
data = Classifier(train, test, tokenizer, classes, max_seq_len=128)

1370it [00:01, 801.61it/s]
<ipython-input-55-1744e357f7d9>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x), np.array(y)
457it [00:00, 797.05it/s]


max seq_len 249


In [ ]:
print(data.test_x.shape)
print(data.test_y.shape)
print(data.train_x.shape)
print(data.train_y.shape)

(457, 128)
(457,)
(1370, 128)
(1370,)


In [ ]:
data.train_x[120]

array([ 101,  207, 1870,  215, 1002,  223,  596,  211,  219,  117,  102,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [ ]:
data.train_x[8]

array([  101, 25430,   245,   117,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 128)]             0         
                                                                 
 tf_bert_model_2 (TFBertMod  TFBaseModelOutputWithPo   109482240 
 el)                         olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 128, 768),                       
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                           

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5, clipvalue=0.5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
history = model.fit(
  x=data.train_x,
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=15,
)

Epoch 1/15
78/78 [==============================] - 1982s 25s/step - loss: 2.6157 - acc: 0.0973 - val_loss: 2.0968 - val_acc: 0.1752
Epoch 2/15
78/78 [==============================] - 1867s 24s/step - loss: 2.3273 - acc: 0.1484 - val_loss: 2.2577 - val_acc: 0.1606
Epoch 3/15
78/78 [==============================] - 1892s 24s/step - loss: 2.1047 - acc: 0.2360 - val_loss: 2.0101 - val_acc: 0.2409
Epoch 4/15
78/78 [==============================] - 1848s 24s/step - loss: 1.7837 - acc: 0.3690 - val_loss: 2.1779 - val_acc: 0.1460
Epoch 5/15
61/78 [======================>.......] - ETA: 6:33 - loss: 1.4746 - acc: 0.4959

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

ax.plot(history.history['loss'],'b')
ax.plot(history.history['val_loss'],'r')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'valid'])
plt.title('Loss over training epochs')
plt.show();

In [ ]:
ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

ax.plot(history.history['acc'],'b')
ax.plot(history.history['val_acc'],'r')
plt.ylabel(' Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'valid'])
plt.title('Accuracy over training epochs')
plt.show();

In [ ]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print("test acc", test_acc)

In [ ]:
y_pred = model.predict(data.test_x).argmax(axis=-1)
y_pred

In [ ]:
print(classification_report(data.test_y,y_pred, target_names=classes))